#  Índice invertido e Busca Booleana 

   Essa é a resolução da primeira parte do Lab 01 da disciplina Recuperação da Informação e Busca na Web. A atividade consiste em primeiro lugar na construção de um índice invertido a partir do arquivo de texto disponibilizado.  
    Por fim, devem ser implementados algoritmos de busca booleana que utilizarão o índice invertido para responder consultas booleanas. Esses algoritmos recebem termos fornecidos pelo usuário para a realização das buscas. Três algoritmos serão exigidos:
* 1-termo: usuário passa um termo e o algoritmo retorna os documentos associados;
* Busca AND: todos os termos da consulta devem estar presentes na página;
* Busca OR: algum dos termos da consulta deve estar presente na página.


In [36]:
import pandas as pd
import numpy as np
import nltk as nl
from collections import defaultdict

    Em primeiro momento vamos conhecer os dados que consiste em um .csv de notícias do Estadão. Temos no arquivo acesso ao título da notícia, o conteúdo da notícia e, por último, o id.

In [37]:
df = pd.read_csv("noticias_estadao.csv", encoding='utf-8')
df

,titulo,conteudo,idNoticia
0,PT espera 30 mil pessoas em festa na Esplanada,BRASÍLIA - Após o desgaste provocado com o lan...,1
1,Alckmin toma posse de olho no Planalto,"Reeleito em outubro, o governador tucano Geral...",2
2,Seis obstáculos e desafios do segundo mandato ...,1. Rearranjo das contas A nova equipe econôm...,3
3,Veja os desafios dos governadores que assumem ...,"No Acre, governador reeleito quer erradicar an...",4
4,PT impulsiona cerimônia de posse da Dilma nas ...,"Os perfis da presidente Dilma Rousseff, nas re...",5
5,Dilma veta projeto de Suplicy que cria linha o...,A presidente Dilma Rousseff vetou integralment...,6
6,Dilma anuncia 14 ministros e conclui reforma m...,A presidente Dilma Rousseff concluiu nesta qua...,7
7,Dilma anuncia últimos 14 ministros e fecha equipe,O Palácio do Planalto anunciou no fim da manhã...,8
8,Ex-governador critica Alckmin pela falta de ág...,O ex-governador e integrante da Executiva do P...,9
9,Veja lista completa de ministros anunciados po...,Veja a lista de ministros já anunciados pela p...,10


Agora, vamos inicializar as buscas.

In [38]:
def busca(termo):
    return um_termo(termo)

In [39]:
def busca(termo1, op, termo2):
    if(op == "AND"):
        return busca_and(termo1, termo2)
    elif(op == "OR"):
        return busca_or(termo1, termo2)

In [71]:
def um_termo(termo):
    return i_indexes[termo]

In [41]:
def busca_and(termo1, termo2):
    ocorr_1 = set(i_indexes[termo1])
    ocorr_2 = set(i_indexes[termo2])
    return ocorr_1 & ocorr_2

In [42]:
def busca_or(termo1, termo2):
    ocorr_1 = set(i_indexes[termo1])
    ocorr_2 = set(i_indexes[termo2])
    return ocorr_1 | ocorr_2

In [53]:
def gera_tokens(df):
    indexes = defaultdict(set)
    for index,row in df.iterrows():
        title_tokens = (word.lower() for word in (nl.word_tokenize(row['titulo'])))
        notice_tokens = (word.lower() for word in (nl.word_tokenize(row['conteudo'])))
        
        for token in title_tokens:
            indexes[token].add(row['idNoticia'])

        for token in notice_tokens:
            indexes[token].add(row['idNoticia'])    
    return indexes
                  

In [54]:
i_indexes = gera_tokens(df)


368
664
906
2463
2595
2671
2690
2725
3018
3100
3207
3313
3394
3430
3467
3533
3598
3685
3758
3860
3924
4267
4315
4384
4437
4505
4539
4624
4699
4794
4799
4849
5084
5109
5176
5222
5453
5520
5586
5620
5620
5638
5957
5990
6070
6117
6161
6217
6270
6367
6412
6585
6596
6644
6669
6706
6740
6748
6804
6816
6869
6984
7010
7045
7108
7155
7208
7298
7349
7362
7402
7515
7561
7577
7578
7610
7659
7734
7800
7825
7856
7863
7865
7921
8029
8068
8102
8107
8142
8186
8215
8229
8279
8298
8380
8424
8478
8508
8509
8537
8593
8618
8663
8728
8848
8903
8943
8961
8978
8982
9019
9065
9085
9159
9263
9293
9337
9343
9388
9419
9450
9487
9513
9522
9552
9586
9641
9681
9739
9794
9821
9871
9889
9890
9913
9954
9964
10520
10595
10648
10703
10716
10747
10845
10909
11045
11070
11086
11115
11187
11203
11227
11242
11245
11388
11412
11431
11497
11571
11605
11627
11662
11665
11698
11717
11742
11765
11798
11918
11955
11971
11993
12034
12085
12109
12143
12148
12175
12191
12356
12371
12406
12437
12495
12508
12521
12533
12540
12574
12593


29858
29889
29904
29924
29924
29924
29926
29928
29933
29943
29945
29951
29951
29957
29958
29960
29964
29969
29991
30025
30045
30046
30047
30057
30064
30072
30085
30099
30104
30109
30116
30123
30128
30132
30142
30156
30162
30170
30170
30170
30177
30177
30215
30227
30235
30245
30245
30270
30290
30292
30296
30296
30297
30301
30313
30314
30322
30323
30328
30339
30348
30368
30389
30392
30394
30395
30401
30409
30416
30417
30420
30425
30433
30435
30436
30438
30441
30442
30442
30453
30458
30463
30467
30476
30517
30537
30542
30568
30568
30588
30588
30595
30606
30611
30620
30626
30627
30633
30641
30641
30646
30649
30656
30665
30677
30680
30684
30693
30695
30702
30703
30704
30704
30712
30723
30724
30727
30732
30732
30754
30757
30758
30772
30782
30791
30810
30829
30848
30851
30851
30878
30885
30890
30896
30900
30906
30908
30912
30918
30930
30939
30942
30951
30954
30962
30971
30974
30975
30975
30983
30997
31002
31005
31007
31008
31012
31012
31015
31018
31018
31018
31029
31059
31062
31102
31107
3112

39857
39865
39868
39871
39874
39879
39882
39882
39883
39885
39890
39890
39894
39894
39903
39915
39934
39936
39944
39945
39948
39952
39962
39967
39971
39974
39978
39981
39981
39981
39987
39988
39990
40001
40001
40001
40011
40014
40016
40023
40026
40040
40041
40041
40041
40045
40048
40049
40051
40051
40056
40058
40059
40099
40100
40102
40109
40117
40117
40124
40131
40134
40137
40137
40145
40145
40147
40149
40158
40164
40168
40170
40177
40177
40177
40193
40193
40193
40201
40206
40209
40212
40221
40221
40223
40232
40234
40237
40238
40240
40240
40244
40266
40269
40275
40275
40276
40277
40284
40290
40292
40298
40304
40308
40316
40336
40339
40352
40367
40370
40374
40375
40379
40380
40392
40394
40395
40395
40403
40407
40411
40423
40434
40438
40439
40447
40462
40462
40469
40470
40474
40475
40476
40478
40527
40531
40549
40560
40561
40571
40571
40575
40591
40591
40592
40592
40594
40599
40599
40642
40652
40682
40685
40692
40703
40706
40713
40716
40716
40720
40726
40729
40737
40745
40746
40750
4075

47313
47314
47321
47324
47328
47330
47339
47343
47344
47345
47346
47353
47356
47361
47364
47368
47368
47376
47391
47398
47398
47405
47411
47421
47426
47427
47433
47436
47437
47441
47442
47442
47443
47444
47446
47447
47448
47449
47450
47464
47470
47470
47472
47480
47481
47484
47486
47488
47494
47494
47498
47503
47508
47511
47512
47514
47539
47548
47550
47552
47555
47555
47559
47562
47565
47569
47569
47592
47602
47629
47632
47633
47636
47636
47640
47641
47663
47670
47676
47688
47691
47712
47720
47731
47736
47741
47743
47778
47783
47805
47816
47816
47820
47822
47822
47826
47826
47827
47829
47833
47834
47834
47841
47850
47859
47862
47864
47864
47868
47869
47876
47878
47898
47929
47968
48056
48065
48067
48087
48095
48116
48123
48129
48149
48151
48159
48172
48177
48179
48181
48184
48185
48186
48192
48208
48214
48219
48233
48242
48244
48250
48254
48256
48259
48261
48264
48264
48265
48265
48266
48270
48295
48303
48306
48316
48319
48322
48324
48327
48329
48332
48333
48335
48337
48338
48341
4834

54250
54250
54251
54252
54255
54263
54264
54266
54269
54273
54281
54283
54284
54296
54298
54298
54298
54304
54316
54317
54322
54324
54380
54390
54392
54395
54395
54396
54396
54413
54418
54420
54421
54421
54423
54427
54428
54432
54438
54441
54451
54465
54468
54473
54478
54479
54484
54489
54491
54492
54496
54496
54501
54509
54512
54520
54527
54530
54533
54533
54533
54536
54537
54537
54539
54542
54557
54600
54602
54603
54607
54607
54626
54626
54635
54643
54649
54649
54653
54656
54658
54658
54658
54658
54658
54659
54661
54669
54673
54676
54677
54677
54677
54680
54682
54683
54722
54725
54741
54741
54741
54742
54748
54774
54776
54781
54782
54783
54791
54791
54791
54793
54798
54801
54803
54807
54814
54816
54816
54821
54836
54837
54849
54851
54856
54859
54870
54872
54873
54873
54877
54877
54877
54878
54884
54885
54886
54890
54891
54891
54893
54911
54916
54928
54930
54931
54932
54932
54932
54946
54953
54972
54975
54978
54978
54996
54998
55001
55003
55005
55007
55012
55012
55016
55025
55025
5502

60668
60672
60672
60687
60687
60691
60691
60691
60693
60702
60721
60726
60732
60745
60748
60748
60759
60763
60765
60766
60776
60776
60778
60780
60785
60793
60794
60798
60800
60803
60809
60810
60811
60812
60816
60818
60820
60830
60831
60832
60840
60845
60846
60848
60850
60850
60854
60855
60856
60860
60862
60865
60865
60866
60868
60910
60917
60918
60919
60923
60923
60924
60925
60928
60931
60934
60939
60945
60949
60954
60956
60958
60958
60961
60962
60963
60963
60985
60985
60986
60992
60994
60994
60996
61001
61001
61003
61006
61010
61018
61020
61034
61036
61040
61041
61044
61045
61099
61108
61109
61109
61110
61110
61115
61117
61118
61122
61124
61130
61136
61136
61139
61143
61147
61155
61165
61173
61174
61186
61189
61193
61205
61206
61207
61207
61211
61219
61223
61230
61247
61248
61251
61251
61251
61254
61258
61260
61265
61275
61278
61282
61285
61288
61290
61292
61293
61293
61296
61305
61306
61306
61308
61308
61310
61316
61318
61319
61321
61323
61332
61332
61360
61361
61364
61373
61374
6137

In [61]:
#arrumar isso
def sanity_check():
    sanity_list = [1952, 4802, 1987, 6694, 5382, 1770, 2763, 1068, 5870, 2777, 1370, 2779]
    result = busca("campina", "AND","grande")
    return result

In [62]:
print(sanity_check())

set([1952L, 4802L, 1987L, 5382L, 6694L, 1770L, 2763L, 1068L, 5870L, 2777L, 1370L, 2779L])


In [ ]:
#nl.download()

In [88]:
print len((um_termo("lula")))
print len(busca("lula", "AND", "dilma"))
print len(busca("lula","OR","dilma"))

1880
1561
4500


In [79]:
assert len(busca("debate","OR","presidencial")) == 1770
assert len(busca("debate","AND","presidencial")) == 201

In [94]:
print  len(busca("presidenciáveis","OR","corruptos"))
print len(um_termo("presidenciáveis"))
assert len(busca("presidenciáveis","OR","corruptos")) == 164


53
0


AssertionError: 

In [82]:
assert len(busca("presidenciáveis","AND","corruptos")) == 0

In [89]:
assert len(busca("belo","OR","horizonte")) == 331
assert len(busca("belo","AND","horizonte")) == 242